In [1]:
import os, sys
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
# Input

#Covarage matrix as csv file without qoutes, seperated by commas
file = "/home/pato/Skrivebord/HPV_subtyping/Andet/covMatrixTest.csv"

# Housegenes to check and remove
housegenes = ['PABPN1','SRSF3','PPIE','RAB1B','BTF3']

# Location of all valid reference names that can be given
path = "/home/pato/Skrivebord/HPV_subtyping/References"

# Location of where to save output files
refSavePath = "/home/pato/Skrivebord/HPV_subtyping/References/InputRefs"

In [3]:
faF = [f for f in listdir(path) if isfile(join(path, f))]
faFiles = []
for i in faF:
    if "fasta.fai" not in i:
        faFiles.append(i)

In [4]:
df = pd.read_csv(file,sep=",")


In [5]:
# Finder samples

excludeCols = ['Sort column', 'Gene', 'contig_srt', 'contig_end', 'Target']
sampleList = []
discardedList = []

for col in df.columns:
  
    if col not in excludeCols:

        # Checking housegenes, first get total cov of them
        hgTot = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            hgTot+= int(hg[col])

        # Now finding outliers in housegene coverage
        notEnoughCov = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            geneCov = int(hg[col])

            # set min cov on housegene to not count as failed
            if geneCov < hgTot*0.1:
                notEnoughCov+=1
            
        # discard sample if too many low housegenes
        if notEnoughCov < 4:
            sampleList.append(col)
        else:
            discardedList.append(col)

In [6]:
discardedList

[]

In [7]:
# Now remove housegenes from samples
ddf = df[~df['Gene'].isin(housegenes)]

In [8]:
# Split i samples og find hvilke HPV typer der passer
hpvTypes = pd.unique(ddf['Gene'])

In [9]:
filtered = ddf[ddf['Gene'] == "HPV45HR_X74479"]
meanCov = filtered["IonXpress_041"].sum()

In [10]:
# For sample, find HPV types in them
covJson = {}
for sample in sampleList:
    covJson.update({sample : {
    }})

    for hpv in hpvTypes:
        filtered = ddf[ddf['Gene'] == hpv]
        
        # Tilsætter sample, hpvtype og sum af dækning for regioner i hpvtype
        covJson[sample].update({
            hpv : filtered[sample].sum()
            })

In [ ]:
# Tæl sum af dækning over alle regioner for sample, derefter angiv fraktioner

ratioJson = {}


for sample in covJson.keys():
    # Tæl total cov over gener for sample
    sampleCovTotal = 0
    for hpvtype, cov in covJson[sample].items():
        sampleCovTotal+=cov
    # Har nu total, og kan derfor loope igennem igen og lave ratios
    # Sætter sample name:
    ratioJson.update({sample : {
        }})
    
    for hpvtype, cov in covJson[sample].items():
        

        ratioJson[sample].update({
            hpvtype : cov/sampleCovTotal*100
            })

In [48]:
confirmedJson = {}


for sample in ratioJson.keys():
    
    confirmedJson.update({sample : {
        }})
    
    foundOneForSample = "no"
    highestRatio = 0
    for hpvtype, ratioCov in ratioJson[sample].items():
        
        # Define minimum ratio to confirm in sample (1 to 100)
        if ratioCov > 20: 
            confirm = "yes" 
            foundOneForSample = "yes"
            if ratioCov > highestRatio:
                highestRatio = ratioCov
                highestRatioName = hpvtype
        else: 
            confirm = "no"

        confirmedJson[sample].update({
            hpvtype : confirm
            })

        # If no one meets the minimum ratioCov value, set the highest cov as best match
        if foundOneForSample == "no":
            for hpvtype, ratioCov in ratioJson[sample].items():
                if hpvtype == highestRatioName:
                    confirmedJson[sample].update({
                        hpvtype : "yes"
                        })




In [51]:
# For each sample output confirmed hpv types to file
multiFa = []
for sample in confirmedJson.keys():
    for hpvtype, val in confirmedJson[sample].items():
        if val == "yes":

            # convert name to fit fasta files:
            ht = hpvtype.split('_')[1]

            # search list of fasta files for id
            ref = [i for i in faFiles if ht in i]
            # Checking that there is only one match, then saving without .fasta in name
            if len(ref) > 1:
                multiFa.append([sample, ref])
            else:
                ref = ref[0].replace(".fasta","")
                # Append to file
                filename = refSavePath+"/"+sample+".txt"
                f = open(filename, "a")
                f.write(ref+"\n")
                f.close()

# Printing those with multiple hits. Fix these by removing one or more of the files
if len(multiFa) > 0:
    print(multiFa)

In [35]:
len(sampleList)

24